In [ ]:
#识别离心阶段

# 加载 YOLOv8 姿态估计模型
model = YOLO('yolov8l-pose.pt')
file_name = 'squat_segment_1.mp4'
# 打开深蹲的分段视频
cap = cv2.VideoCapture(file_name)  # 使用你的深蹲分段视频路径
fps = int(cap.get(cv2.CAP_PROP_FPS))  # 获取帧率
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 定义视频输出格式
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# 定义变量
previous_position = None
tracking_ascending = False  # 标记是否进入向心阶段
movement_threshold = 8  # 设置检测向上移动的差阈值为 15 像素
video_out = None  # 用于保存剪辑的视频

frame_number = 0

# 遍历视频的每一帧
results = model(file_name)  # 使用 YOLO 模型进行姿态检测

for result in results:
    ret, frame = cap.read()
    if not ret:
        break

    # 获取关键点
    all_keypoints = result.keypoints.data  # 所有关键点

    max_area = 0
    largest_person = None

    # 找到检测到的最大目标
    for keypoints in all_keypoints:
        area = calculate_bounding_box_area(keypoints)
        if area > max_area:
            max_area = area
            largest_person = keypoints

    if largest_person is not None:
        # 计算左肩膀、左髋关节、左膝盖的坐标平均值，作为检测的垂直位置
        avg_position = (largest_person[11][:2].cpu().numpy() +  # 左髋关节
                        largest_person[5][:2].cpu().numpy()) / 2  # 左肩膀

        # 如果没有初始化上一帧的位置，先初始化
        if previous_position is None:
            previous_position = avg_position
            frame_number += 1
            continue

        # 计算平均坐标的垂直移动量（Y 轴）
        vertical_movement = previous_position[1] - avg_position[1]  # 如果位置上升，垂直位移为正

        # 检测向心阶段的开始
        if not tracking_ascending and vertical_movement > movement_threshold:
            # 向心阶段开始
            print(f"Detected start of concentric phase at frame {frame_number}.")
            tracking_ascending = True
            # 开始保存视频
            video_out = cv2.VideoWriter(f'concentric_segment_{frame_number}.mp4', fourcc, fps, (width, height))

        # 如果进入了向心阶段，保存视频
        if tracking_ascending:
            video_out.write(frame)

            

        # 更新上一帧的位置
        previous_position = avg_position

    frame_number += 1
if video_out is not None:
    video_out.release()  # 结束视频片段保存
# 释放资源
cap.release()
if video_out is not None:
    video_out.release()